In [1]:
!pip3 install -r requirements.txt

  Using cached aniso8601-8.1.0-py2.py3-none-any.whl (44 kB)
  Using cached ansi2html-1.6.0-py3-none-any.whl (14 kB)
  Using cached arrow-0.17.0-py2.py3-none-any.whl (50 kB)
  Using cached asttokens-2.0.4-py2.py3-none-any.whl (20 kB)
  Using cached attrs-20.3.0-py2.py3-none-any.whl (49 kB)
  Using cached certifi-2020.12.5-py2.py3-none-any.whl (147 kB)
  Using cached chardet-4.0.0-py2.py3-none-any.whl (178 kB)
  Using cached click-7.1.2-py2.py3-none-any.whl (82 kB)
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
  Using cached Cython-0.29.21-cp36-cp36m-manylinux1_x86_64.whl (2.0 MB)
  Using cached executing-0.5.4-py3-none-any.whl (12 kB)
  Using cached Flask-1.1.2-py2.py3-none-any.whl (94 kB)
  Using cached Flask_RESTful-0.3.8-py2.py3-none-any.whl (25 kB)
  Using cached idna-2.10-py2.py3-none-any.whl (58 kB)
  Using cached importlib_metadata-3.3.0-py3-none-any.whl (10 kB)
  Using cached iniconfig-1.1.1-py2.py3-none-any.whl (5.0 kB)
  Using cached itsdangerous-1.1.0-py2.py3-non

In [1]:
import os
from utilities.data_utils.Dataset import FacialDataset, get_transform
from utilities.utils import collate_fn
from utilities.train_eval.engine import train_one_epoch, evaluate, get_model_result
import glob

import nvidia_smi # for python 3, you need nvidia-ml-py3 library

import torch
torch.cuda.empty_cache()
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.rpn import AnchorGenerator, RPNHead

In [2]:
PATH = 'Weight.pth'

output_image_folder = 'output'

proj = 'fabric' # 'sign','fabric' (put ur project name here)

if proj == 'sign':
    classes = ['warning', 'prohibitory', 'mandatory']
elif proj == 'fabric':
    classes = ['Hole', 'Line', 'Stain']
else:
    classes = ['bg']

num_classes = len(classes)+1  # n class + background

batch_size = 3

num_epochs = 30

In [3]:
torch.cuda.empty_cache()
nvidia_smi.nvmlInit()
handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
print("Total memory:", info.total)
print("Free memory:", info.free)
print("Used memory:", info.used)

Total memory: 8505131008
Free memory: 8205434880
Used memory: 299696128


In [4]:
dataset_train = FacialDataset('data/train', get_transform(horizontal_flip=True),classes=classes)
dataset_test = FacialDataset('data/test', get_transform(horizontal_flip=False),classes=classes)

data_loader_train = torch.utils.data.DataLoader(
        dataset_train, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=0,
        collate_fn=collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=0,
    collate_fn=collate_fn)

In [5]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)


In [6]:
anchor_generator = AnchorGenerator(sizes=((32,), (24, ), (24, ), (16,), (8, )),
                                        aspect_ratios=([1.0, 1.0, 1.0, 1.0], 
                                                     [0.8, 1.0, 1.0, 1.0], 
                                                     [1.0, 0.8, 1.0, 1.0],
                                                     [1.0, 1.0, 1.0, 1.0],
                                                     [1.0, 1.0, 1.0, 1.0]))

In [7]:
model.rpn.anchor_generator = anchor_generator
model.rpn.head = RPNHead(256, anchor_generator.num_anchors_per_location()[0])
# get the number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)


In [8]:
if os.path.isfile(PATH):
    model.load_state_dict(torch.load(PATH))

model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

## Training

In [9]:
for epoch in range(num_epochs):
    res = nvidia_smi.nvmlDeviceGetUtilizationRates(handle)
    print(f'gpu: {res.gpu}%, gpu-mem: {res.memory}%')

    train_one_epoch(model, optimizer, data_loader_train, device, epoch, print_freq=1)
    lr_scheduler.step()
    evaluate(model, data_loader_test, device=device)

print("Training complete!")

gpu: 0%, gpu-mem: 0%
Epoch: [0]  [  0/128]  eta: 0:16:01  lr: 0.000044  loss: 5.8745 (5.8745)  loss_classifier: 1.8842 (1.8842)  loss_box_reg: 0.0013 (0.0013)  loss_objectness: 0.6939 (0.6939)  loss_rpn_box_reg: 3.2951 (3.2951)  time: 7.5125  data: 3.0859  max mem: 3398
Epoch: [0]  [  1/128]  eta: 0:09:37  lr: 0.000084  loss: 5.6608 (5.7677)  loss_classifier: 1.8842 (1.9046)  loss_box_reg: 0.0013 (0.0014)  loss_objectness: 0.6920 (0.6929)  loss_rpn_box_reg: 3.0422 (3.1687)  time: 4.5497  data: 1.6799  max mem: 3669
Epoch: [0]  [  2/128]  eta: 0:07:20  lr: 0.000123  loss: 5.6608 (5.4069)  loss_classifier: 1.8842 (1.8726)  loss_box_reg: 0.0013 (0.0010)  loss_objectness: 0.6939 (0.6940)  loss_rpn_box_reg: 3.0422 (2.8392)  time: 3.4923  data: 1.1279  max mem: 3669
Epoch: [0]  [  3/128]  eta: 0:06:11  lr: 0.000162  loss: 5.6608 (6.0348)  loss_classifier: 1.8086 (1.7817)  loss_box_reg: 0.0013 (0.0012)  loss_objectness: 0.6928 (0.6937)  loss_rpn_box_reg: 3.0422 (3.5582)  time: 2.9738  data: 0

Epoch: [0]  [ 33/128]  eta: 0:02:35  lr: 0.001342  loss: 3.8041 (4.3720)  loss_classifier: 0.0677 (0.3896)  loss_box_reg: 0.0027 (0.0065)  loss_objectness: 0.6909 (0.6918)  loss_rpn_box_reg: 2.9025 (3.2840)  time: 1.4740  data: 0.1380  max mem: 3687
Epoch: [0]  [ 34/128]  eta: 0:02:33  lr: 0.001382  loss: 3.7007 (4.3430)  loss_classifier: 0.0677 (0.3865)  loss_box_reg: 0.0027 (0.0071)  loss_objectness: 0.6907 (0.6916)  loss_rpn_box_reg: 2.8528 (3.2578)  time: 1.4750  data: 0.1403  max mem: 3687
Epoch: [0]  [ 35/128]  eta: 0:02:31  lr: 0.001421  loss: 3.6998 (4.3155)  loss_classifier: 0.0773 (0.3816)  loss_box_reg: 0.0044 (0.0071)  loss_objectness: 0.6896 (0.6915)  loss_rpn_box_reg: 2.7147 (3.2353)  time: 1.4749  data: 0.1349  max mem: 3687
Epoch: [0]  [ 36/128]  eta: 0:02:29  lr: 0.001460  loss: 3.6998 (4.3224)  loss_classifier: 0.0843 (0.3735)  loss_box_reg: 0.0049 (0.0071)  loss_objectness: 0.6895 (0.6914)  loss_rpn_box_reg: 2.7147 (3.2504)  time: 1.4830  data: 0.1394  max mem: 3687


Epoch: [0]  [ 66/128]  eta: 0:01:37  lr: 0.002640  loss: 2.8378 (3.7721)  loss_classifier: 0.0613 (0.2434)  loss_box_reg: 0.0205 (0.0137)  loss_objectness: 0.6816 (0.6877)  loss_rpn_box_reg: 2.1249 (2.8272)  time: 1.5003  data: 0.1334  max mem: 3687
Epoch: [0]  [ 67/128]  eta: 0:01:35  lr: 0.002679  loss: 2.5713 (3.7495)  loss_classifier: 0.0613 (0.2407)  loss_box_reg: 0.0241 (0.0139)  loss_objectness: 0.6816 (0.6875)  loss_rpn_box_reg: 1.8096 (2.8074)  time: 1.5033  data: 0.1366  max mem: 3687
Epoch: [0]  [ 68/128]  eta: 0:01:34  lr: 0.002719  loss: 2.5713 (3.7423)  loss_classifier: 0.0588 (0.2378)  loss_box_reg: 0.0205 (0.0139)  loss_objectness: 0.6816 (0.6875)  loss_rpn_box_reg: 1.8096 (2.8031)  time: 1.5005  data: 0.1381  max mem: 3687
Epoch: [0]  [ 69/128]  eta: 0:01:32  lr: 0.002758  loss: 2.5713 (3.7346)  loss_classifier: 0.0588 (0.2355)  loss_box_reg: 0.0241 (0.0141)  loss_objectness: 0.6805 (0.6874)  loss_rpn_box_reg: 1.8096 (2.7976)  time: 1.4956  data: 0.1316  max mem: 3687


Epoch: [0]  [ 99/128]  eta: 0:00:45  lr: 0.003938  loss: 2.4536 (3.4321)  loss_classifier: 0.0888 (0.1936)  loss_box_reg: 0.0565 (0.0282)  loss_objectness: 0.6426 (0.6762)  loss_rpn_box_reg: 1.7173 (2.5341)  time: 1.5275  data: 0.1462  max mem: 3687
Epoch: [0]  [100/128]  eta: 0:00:43  lr: 0.003977  loss: 2.4258 (3.4192)  loss_classifier: 0.0888 (0.1921)  loss_box_reg: 0.0565 (0.0281)  loss_objectness: 0.6364 (0.6752)  loss_rpn_box_reg: 1.5836 (2.5238)  time: 1.5292  data: 0.1439  max mem: 3687
Epoch: [0]  [101/128]  eta: 0:00:41  lr: 0.004017  loss: 2.4258 (3.4286)  loss_classifier: 0.0880 (0.1910)  loss_box_reg: 0.0565 (0.0287)  loss_objectness: 0.6364 (0.6749)  loss_rpn_box_reg: 1.5836 (2.5339)  time: 1.5319  data: 0.1537  max mem: 3687
Epoch: [0]  [102/128]  eta: 0:00:40  lr: 0.004056  loss: 2.4258 (3.4234)  loss_classifier: 0.0888 (0.1911)  loss_box_reg: 0.0565 (0.0303)  loss_objectness: 0.6324 (0.6745)  loss_rpn_box_reg: 1.5836 (2.5275)  time: 1.5539  data: 0.1750  max mem: 3688


Epoch: [1]  [  0/128]  eta: 0:07:20  lr: 0.005000  loss: 3.3970 (3.3970)  loss_classifier: 0.1297 (0.1297)  loss_box_reg: 0.1030 (0.1030)  loss_objectness: 0.5032 (0.5032)  loss_rpn_box_reg: 2.6610 (2.6610)  time: 3.4447  data: 0.1958  max mem: 3688
Epoch: [1]  [  1/128]  eta: 0:07:01  lr: 0.005000  loss: 2.6862 (3.0416)  loss_classifier: 0.1083 (0.1190)  loss_box_reg: 0.0940 (0.0985)  loss_objectness: 0.4801 (0.4916)  loss_rpn_box_reg: 2.0038 (2.3324)  time: 3.3174  data: 0.1161  max mem: 3688
Epoch: [1]  [  2/128]  eta: 0:07:00  lr: 0.005000  loss: 2.6862 (2.7236)  loss_classifier: 0.1083 (0.0993)  loss_box_reg: 0.0940 (0.0768)  loss_objectness: 0.4949 (0.4927)  loss_rpn_box_reg: 2.0038 (2.0549)  time: 3.3408  data: 0.1226  max mem: 3688


KeyboardInterrupt: 

In [10]:
# create output directory
# if output directory exists, delete existing files
if not os.path.exists(output_image_folder):
    os.mkdir(output_image_folder)
else:
    files = glob.glob(output_image_folder + '/*')
    for f in files:
        os.remove(f)

## Evaluate

In [11]:
evaluate(model, data_loader_test, device=device)

n_threads =  1
creating index...
index created!
Test:  [ 0/24]  eta: 0:00:07  model_time: 0.2824 (0.2824)  evaluator_time: 0.0031 (0.0031)  time: 0.2921  data: 0.0052  max mem: 3688
Test:  [23/24]  eta: 0:00:00  model_time: 0.5258 (0.5086)  evaluator_time: 0.0021 (0.0052)  time: 0.5365  data: 0.0111  max mem: 3688
Test: Total time: 0:00:12 (0.5259 s / it)
Averaged stats: model_time: 0.5258 (0.5086)  evaluator_time: 0.0021 (0.0052)
Accumulating evaluation results...
DONE (t=0.02s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.033
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.142
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.004
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.090
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.029
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.010
 Average Recall  

In [12]:
torch.cuda.synchronize()
# create directory for saving the model
print("Saving model...")
torch.save(model.state_dict(), PATH)
print("Model saving complete!")
nvidia_smi.nvmlShutdown()

Saving model...
Model saving complete!


## Inference 
Repeating all the process to ensure nothing is taken from training stage
Check output folder for results

In [14]:
torch.cuda.empty_cache()
nvidia_smi.nvmlInit()
handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
print("Total memory:", info.total)
print("Free memory:", info.free)
print("Used memory:", info.used)

dataset_test = FacialDataset('data/test', get_transform(horizontal_flip=False),classes=classes)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=0,
    collate_fn=collate_fn)


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

anchor_generator = AnchorGenerator(sizes=((32,), (24, ), (24, ), (16,), (8, )),
                                   aspect_ratios=([1.0, 1.0, 1.0, 1.0],
                                                  [0.8, 1.0, 1.0, 1.0],
                                                  [1.0, 0.8, 1.0, 1.0],
                                                  [1.0, 1.0, 1.0, 1.0],
                                                  [1.0, 1.0, 1.0, 1.0]))
model.rpn.anchor_generator = anchor_generator
model.rpn.head = RPNHead(256, anchor_generator.num_anchors_per_location()[0])
# get the number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

model.load_state_dict(torch.load(PATH))
model.to(device)

# create output directory
# if output directory exists, delete existing files
if not os.path.exists(output_image_folder):
    os.mkdir(output_image_folder)
else:
    files = glob.glob(output_image_folder + '/*')
    for f in files:
        os.remove(f)
# write testing result to output folder
for img_idx, batch_sampler in enumerate(data_loader_test):
    img_test = batch_sampler[0][0]
    target_test = batch_sampler[1][0]
    i = target_test["image_id"].item()
    get_model_result(img_test, model, target_test, i, device, location=output_image_folder, threshold=0.2,classes=classes)

print("Testing complete!")

torch.cuda.synchronize()
nvidia_smi.nvmlShutdown()

Total memory: 8505131008
Free memory: 6725173248
Used memory: 1779957760
initial boxes: ['Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Stain', 'Hole', 'Hole', 'Stain', 'Hole', 'Hole', 'Hole', 'Stain', 'Stain', 'Stain', 'Hole', 'Hole', 'Hole', 'Stain', 'Hole', 'Stain', 'Stain', 'Stain', 'Hole', 'Hole', 'Stain', 'Hole', 'Stain', 'Stain', 'Stain', 'Stain', 'Stain', 'Line', 'Stain', 'Stain', 'Stain', 'Stain', 'Stain', 'Hole', 'Hole', 'Hole', 'Hole']
initial boxes: ['Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Stain', 'Hole', 'Hole', 'Hole', 'Hole', 'Stain', 'Stain', 'Hole', 'Stain', 'Stain', 'Stain', 'Stain', 'Hole', 'Hole', 'Stain', 'Stain', 'Hole', 'Stain', 'Hole', 'Line', 'Stain', 'Hole', 'Stain', 'Stain', 'Hole', 'Stain', 'Hole', 'Stain', 'Hole', 'Stain', 'Line', 'Hole', 'Stain', 'Hole', 'Line', 'Hole', 'Hole', 'Hole', 'Stain']
initial boxes: ['Hole', 'Hole', 'Hole', 'Hole', 'Hole', '

initial boxes: ['Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Stain', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Stain', 'Hole', 'Stain', 'Stain', 'Stain', 'Stain', 'Hole', 'Stain', 'Hole', 'Hole', 'Hole', 'Stain', 'Stain', 'Stain', 'Hole', 'Stain', 'Stain', 'Stain', 'Stain', 'Stain', 'Hole', 'Hole', 'Stain', 'Stain', 'Stain', 'Hole', 'Stain', 'Hole', 'Stain', 'Stain', 'Stain', 'Line', 'Hole', 'Stain', 'Line', 'Line', 'Hole', 'Hole', 'Line', 'Line', 'Stain', 'Stain', 'Stain', 'Line', 'Stain', 'Hole', 'Hole', 'Hole', 'Stain', 'Hole', 'Hole', 'Stain', 'Stain', 'Stain', 'Hole', 'Hole', 'Hole', 'Stain', 'Stain']
initial boxes: ['Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Hole',